In [11]:
import pandas as pd
import numpy as np
import datetime
import json
import requests
import time
import sys

    
from objsize import get_deep_size
from pympler import asizeof


In [2]:
url = "http://gbfs.citibikenyc.com/gbfs/gbfs.json"

data_urls = requests.get(url).json()
station_status_url =data_urls['data']['en']['feeds'][2]['url']

# Inspect single station data

In [3]:
station_status = requests.get(station_status_url).json()

a_station = station_status['data']['stations'][0]
a_station['station_id']
a_station['last_reported']
a_station
a_station 

{'num_bikes_disabled': 0,
 'last_reported': 86400,
 'is_installed': 0,
 'station_id': '72',
 'legacy_id': '72',
 'is_renting': 0,
 'eightd_has_available_keys': False,
 'num_ebikes_available': 0,
 'num_docks_disabled': 0,
 'is_returning': 0,
 'station_status': 'out_of_service',
 'num_bikes_available': 0,
 'num_docks_available': 0}

# Instantiate the data dictionary

In [4]:
data_dict = {}
for station in station_status['data']['stations']:
    station_id = station['station_id']
    station_time = station['last_reported']
    
    subsubdict = {'num_bikes_available': station['num_bikes_available'], 
           'num_ebikes_available': station['num_ebikes_available'], 
            }
    subdict = {station_time: subsubdict}
    data_dict[station_id] = subdict 

# Request station data iteratively

In [12]:
for i in range(100):
    station_status = requests.get(station_status_url).json()
    
    for station in station_status['data']['stations']:
         
        station_id = station['station_id']
        station_time = station['last_reported']
        if station_time not in data_dict[station_id]:
            subsubdict = {'num_bikes_available': station['num_bikes_available'], 
                   'num_ebikes_available': station['num_ebikes_available'], 
                    }
            subdict = {station_time: subsubdict}
            olddict = data_dict[station_id]
            data_dict[station_id] = {**olddict, **subdict}
    time.sleep(1)
    
    if i%10 == 0: 
#         print(f"iteration: {i}, data_dict size: {data_dict.__sizeof__()}" )
        print(f"iteration: {i}, data_dict size: {get_deep_size(data_dict)}" )
        print(f"iteration: {i}, data_dict size: {asizeof.asizeof(data_dict)}" )
     
        
#         dic3.__sizeof__()

iteration: 0, data_dict size: 702624
iteration: 0, data_dict size: 763528
iteration: 10, data_dict size: 711064
iteration: 10, data_dict size: 771968
iteration: 20, data_dict size: 721216
iteration: 20, data_dict size: 782120
iteration: 30, data_dict size: 730184
iteration: 30, data_dict size: 791088
iteration: 40, data_dict size: 737832
iteration: 40, data_dict size: 798736
iteration: 50, data_dict size: 746560
iteration: 50, data_dict size: 807464
iteration: 60, data_dict size: 755136
iteration: 60, data_dict size: 816040
iteration: 70, data_dict size: 766880
iteration: 70, data_dict size: 827784
iteration: 80, data_dict size: 779160
iteration: 80, data_dict size: 840064
iteration: 90, data_dict size: 804540
iteration: 90, data_dict size: 865448


NameError: name 'my_object' is not defined

# Inspect dictionary

In [16]:
for key, item in data_dict['247'].items():
    print(datetime.datetime.fromtimestamp(key) )

2020-08-12 20:39:54


In [17]:
dict_of_df = {k: pd.DataFrame(v) for k,v in data_dict.items()}
df = pd.concat(dict_of_df, axis=1).transpose()
df

num_bikes_available  num_ebikes_available
72   86400                         0                     0
79   1597278788                   18                     0
     1597279401                   17                     0
82   1597278799                   17                     0
83   1597278877                   40                     0
...                              ...                   ...
4109 86400                         0                     0
4112 86400                         0                     0
4113 86400                         0                     0
4114 86400                         0                     0
4115 86400                         0                     0

[1960 rows x 2 columns]

# Save dictionary

In [13]:
with open("station_data.json", "w") as outfile:  
    json.dump(data_dict, outfile)  

In [22]:
now = datetime.datetime.now()

In [20]:
tomorrow_noon = datetime.datetime(2020,8,13,12,0,0)

In [33]:
dt = tomorrow_noon - now
# dt.days
dt.seconds / 3600/24 /7

0.08987929894179894

In [31]:
i = 0

while True:
    i = i+1
    
    print(i)
    if i >=10:
        break

1
2
3
4
5
6
7
8
9
10
